In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../utils"))
from aoc_utils import load_data, check

In [ ]:
import math
import re
from collections import defaultdict
from itertools import product
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
data = load_data(2024, 14)

In [ ]:
# data, part_1, part_2
tests = [
    (
        """p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
""",
        12,
        None,
    ),
    (
        """p=4,5 v=10,0
p=2,2 v=6,5
p=6,1 v=7,-5
p=2,5 v=6,5
p=10,5 v=9,-6
p=10,1 v=4,-1
p=9,4 v=-8,4
p=4,0 v=-7,6
p=4,2 v=0,2
p=5,0 v=10,6
p=7,5 v=-4,5
p=1,3 v=5,2
p=4,3 v=0,-2
p=6,4 v=6,4
p=6,2 v=-9,-6
p=8,4 v=2,-6
p=4,3 v=6,-1
p=1,2 v=-2,3
p=2,0 v=-6,-3
p=0,1 v=-2,6
p=8,4 v=3,-5
p=7,6 v=-4,-4
p=3,5 v=1,1
p=8,1 v=-1,1
p=4,6 v=1,-4
p=8,4 v=-2,2
p=9,0 v=-8,-4
p=3,5 v=7,1
p=0,5 v=10,4
p=7,2 v=2,2
p=3,0 v=0,-5
p=6,2 v=-9,-3
p=5,4 v=-5,4
p=2,5 v=-7,-1
""",
        2016,
        42,
        # a small tree
    ),
]

# Part 1

In [ ]:
def get_robots(data, dims):
    dx, dy = dims
    robots = defaultdict(list)
    lcm = 1
    for line in data.splitlines():
        px, py, vx, vy = (int(v) for v in re.findall(r"(-?\d+)", line))
        robots[px, py].append((vx, vy))
        lcm = math.lcm(lcm, dx // math.gcd(vx, dx))
        lcm = math.lcm(lcm, dy // math.gcd(vy, dy))
    return robots, lcm

In [ ]:
def move(robots, dims, steps=1):
    dx, dy = dims
    future = defaultdict(list)
    for px, py in robots:
        for vx, vy in robots[px, py]:
            x = (px + vx * steps) % dx
            y = (py + vy * steps) % dy
            future[x, y].append((vx, vy))
    return future

In [ ]:
def get_quadrants(data, dims=(101, 103)):
    dx, dy = dims
    midx = dx // 2
    midy = dy // 2
    quadrants = {(i, j): 0 for i, j in product([True, False], repeat=2)}
    robots, _ = get_robots(data, dims)
    robots = move(robots, dims, 100)
    for px, py in robots:
        if px != midx and py != midy:
            quadrants[px > midx, py > midy] += len(robots[px, py])
    return math.prod(quadrants.values())

In [ ]:
check(get_quadrants, tests, dims=(11, 7))
get_quadrants(data)

# Part 2

I didn't really know what to look for.  
First, I tried to identify some horizontal symmetry, thinking the whole area would look like a large, centered tree. It didn't work.

I believe there are (at least) two valid solutions with AOC's inputs:

1. Finding when robots have distinct positions, but there is nothing in the puzzle description that suggests this might be true.
2. Having the most robots forming continuous shapes (filled or not) in the picture, assuming the Christmas tree looks like this:

~~~
...3...    ...3...
..111..    ..1.1..
.12111.    .1...1.
1111121    1111121
..111..    ..1.1..
~~~

I kept the second.

In [ ]:
def adjacency(robots):
    """Count robot positions with at least two distinct neighbors."""
    adjs = 0
    for px, py in robots:
        neighbors = 0
        for dx, dy in product((-1, 0, 1), repeat=2):
            if (px + dx, py + dy) in robots:
                neighbors += 1
        # neighbors include self
        if neighbors >= 3:
            adjs += len(robots[px, py])
    return adjs

In [ ]:
def display_map(robots, dims):
    dx, dy = dims
    picture = np.zeros((dy, dx))
    for x in range(dx):
        for y in range(dy):
            if (x, y) in robots:
                picture[y, x] = len(robots[x, y])
    plt.figure(figsize=(dx / 20, dy / 20))
    plt.imshow(picture)
    plt.axis("off")

In [ ]:
def find_best_picture(data, dims=(101, 103), *, display_picture=True):
    dx, dy = dims
    robots, lcm = get_robots(data, dims=dims)
    assert robots == move(robots, dims, lcm)
    max_adjacency = adjacency(robots)
    best_picture = robots
    when = 0
    for t in range(1, lcm):
        robots = move(robots, dims)
        if (adj := adjacency(robots)) > max_adjacency:
            max_adjacency = adj
            best_picture = robots
            when = t
    if display_picture:
        display_map(best_picture, dims)
    return when

In [ ]:
check(find_best_picture, tests, 2, dims=(11, 7))
find_best_picture(data)